## Export DevInfo Table to Shapefiles

DevInfo stores the three files (.shp, .shx, and .dbf) in 3 columns in the Access database. This script will execute a query that will return those files, plus an additional column, `Layer_NId`, that will be used to construct the file name. This file name will be referenced by the Export to GeoJSON scripts to include the geometry.

In [ ]:
import os
import pyodbc
# [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]
# if you see [], you may need to install the 64-bit ACE drivers
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
# 64bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255

from IPython.display import clear_output

## Configure your paths and folders

In [ ]:
access_database = r'Z:\dev\nepal-import\NepalInfo2016_for_python.accdb'
output_base = 'nepal'
output_shapes_folder = 'shapes'

## Execute the query against the DevInfo Access tables

In [ ]:
connStr = (
    r'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};'
    r'DBQ={};'.format(access_database)
)

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT UT_Area_Map_Layer.Layer_Shp, UT_Area_Map_Layer.Layer_Shx, UT_Area_Map_Layer.Layer_dbf, UT_Area_Map_Layer.Layer_NId, UT_Area_Map_Metadata_en.Layer_Name, UT_Area_en.Area_Parent_NId, UT_Area_en.Area_ID, UT_Area_en.Area_Level, UT_Area_en.Area_Parent_NId
FROM (UT_Area_Map_Layer INNER JOIN UT_Area_Map_Metadata_en ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map_Metadata_en.Layer_NId) INNER JOIN (UT_Area_en INNER JOIN UT_Area_Map ON UT_Area_en.Area_NId = UT_Area_Map.Area_NId) ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map.Layer_NId;
"""
crsr = cnxn.execute(sql)

# export just a subset for testing
# rows = crsr.fetchmany(50)

# uncomment this to get crazy and export them all!!
rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

## Create Shapefiles
Loop through each row in the result set and write out each of the data in the columns to the three files needed for a Shapefile

In [ ]:
row_count = len(rows)

layer_checks = []

for i, row in enumerate(rows):
    
    shp = row[0]
    shx = row[1]
    dbf = row[2]

    layer_id = row[3]

    # We only need to write out one shapefile per layer, so we can skip any duplicates by checking for already created layers
    if layer_id in layer_checks:
        continue

    layer_checks.append(layer_id)

    out_file = '{}/{}'.format(os.path.join(output_base, output_shapes_folder), layer_id)

    print ('adding {} of {}'.format(i+1, row_count))
    clear_output(wait=True)

    with open('{}.shp'.format(out_file), 'wb') as writer:
        writer.write(shp)
    with open('{}.shx'.format(out_file), 'wb') as writer:
        writer.write(shx)
    with open('{}.dbf'.format(out_file), 'wb') as writer:
        writer.write(dbf)
        
    # here we are also writing a GCS WGS 1984 .prj file that will define the spatial reference of each shapefile
    with open('{}.prj'.format(out_file), 'w') as writer:
        writer.write('GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]')